# Predicting Blight Violations

_This project was completed as a part of the Applied Data Science with Python Specialization from Coursera._

In this project, we will evaluate the performance and predictive power of a model that has been trained and tested on data collected from Blight Violation Notices (BVN), or Blight Tickets, that have been issued to property owners who have violated City of Detroit ordinances that govern how property owners must maintain the exterior of their property. Blight Tickets are issued by city inspectors, police officers, neighborhood city hall managers and other city officials who investigate complaints of blight and are managed by the Department of Administrative Hearings.

The target variable is compliance, which is __True__ if the ticket was paid early, on time, or within one month of the hearing data, __False__ if the ticket was paid after the hearing date or not at all, and __Null__ if the violator was found not responsible. Compliance is not avaliable in the dataset so it must be calculated.

The dataset for this project originates from the City of Detroit's Open Data Portal initiative and is updated daily. For this project the dataset spans tickets issued from March 2004 to March 2018. The dataset is avaliable at: https://data.detroitmi.gov/Property-Parcels/Blight-Violations/ti6p-wcg4

## Pre-Processing

Imports the raw data, calculates compliance for each ticket, removes features that cause data leakage, and seperates the data into two dataset based on when the violation occured. Violations issused before 2017 will be used to train and test the model, and tickets issued on and after 2017 will be used to validate the model.

In [1]:
# Common imports
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import normalize
from scipy.sparse import coo_matrix, hstack

# To make the notebook's output stable across runs
random_seed = 12062017
np.random.seed(random_seed)

In [2]:
def load_blight_raw(blight_path):
    '''
    Loads the raw blight_violations.csv
    Returns a pandas dataframe.
    '''
    csv_path = os.path.join(blight_path, "Blight_Violations.csv")
    blight_raw = pd.read_csv(csv_path, encoding='ISO-8859-1')
    print("RAW Blight Violations dataset has {} observations with {} features each.".format(*blight_raw.shape))
    return blight_raw

def pre_processing(blight_df):
    ''' 
    Takes blight panda dataframe and makes column names more reabable, sets ticket_id as index, creates compliance (the 
    prediction variable), and creates compliance_details (explains why a tickets was labeled as complantent or non-complantent).
    Returns a panda dataframe.
    '''
    # Makes column names more reabable
    # Gets columns from dataframe
    columns_names = pd.Series(blight_df.columns)
    # Removes parentheses, removes right spaces
    columns_names = columns_names.str.split('(').str[0].str.strip()
    # Coverts column names to lowercase and replaces spaces with underscores
    columns_names = columns_names.str.lower().str.replace(' ', '_')
    blight_df.columns = columns_names
    # Sets ticket_id as index
    blight_df.set_index('ticket_id', inplace=True)
    # Defines prediction variable compliance and helper variable compliance_detail
    # The values of these variables are NOT correct
    blight_df['compliance'] = 0
    blight_df['compliance_detail'] = np.NaN    
    return blight_df

def clean_up(blight_df):
    # Removes instances where 'ticket_issued_date' is not in (2000, 2017) 
    blight_df = blight_df[blight_df['violation_date'].str.contains('[0-9]{2}/[0-9]{2}/[20]{2}[0-9]{2}') == True]
    # Converts 'ticket_issued_date' to datetime
    blight_df.loc[:, 'violation_date'] = pd.to_datetime(blight_df['violation_date'])
    # Converts 'payment_date' to datetime
    blight_df.loc[:, 'payment_date'] = pd.to_datetime(blight_df['payment_date'].str.extract('([0-9]{2}/[0-9]{2}/20{1}[0-9]{2})')) 
    # Converts bad values in 'ticket_issued_time' to NaNs (Dataset error removed by publisher)
    #blight_df.loc[blight_df['violation_date'] == '00000000000000.000', 'violation_date'] = np.nan
    # Converts 'ticket_issued_time' to datetime.time
    blight_df.loc[:, 'ticket_issued_time'] = pd.DatetimeIndex(blight_df['ticket_issued_time']).time
    # Converts 'hearing_time' to datetime.time
    blight_df.loc[:, 'hearing_time'] = pd.DatetimeIndex(blight_df['hearing_time']).time
    # Remove "$" from 'judgement_amount'
    blight_df.loc[:, 'judgment_amount'] = blight_df['judgment_amount'].str.strip('$')    
    # Converts 'judgment_amount' to float    
    blight_df.loc[:, 'judgment_amount'] = blight_df['judgment_amount'].astype(float)
    # Removes "$" from 'payment_amount' and converts it to float
    blight_df.loc[:, 'payment_amount'] = blight_df['payment_amount'].str.strip('$').astype(float)
    return blight_df

def null_compliance(blight_df):
    '''
    Compliance = np.NaN
    Tickets that cannot be classified as compliant or non-compliant because they were ruled as not responsible in disposition
    or they are awaiting judgement.
    Returns a pandas dataframe
    '''
    # Dispositions that are ruled as not responsible or are still pending
    null_dispositions = ['Not responsible by Dismissal', 'Not responsible by City Dismissal', 'PENDING JUDGMENT', 
                 'Not responsible by Determination','SET-ASIDE (PENDING JUDGMENT)']
    # Loops over dispositions and sets 'compliance' values to np.NaN
    # Loops over dispositions and sets 'compliance_detail' values to 'Not Responsible/Pending Judgement'
    for dispositions in null_dispositions:
        blight_df.loc[blight_df['disposition'] == dispositions, 'compliance'] = np.NaN
        blight_df.loc[blight_df['disposition'] == dispositions, 'compliance_detail'] = 'Not Responsible/Pending Judgement'
    return blight_df

def compliant(blight_df):
    '''
    Compliance = 1
    Tickets that are classified as compliant because they had no fine, fine was waved, made a payment with hearing pending,
    early payment (hearing not pending), payment on time, or payment within one month after hearing date.
    Returns a pandas dataframe
    '''
    ## Compliant by no fine
    # Fine Waived by Determintation
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Determination', 'compliance'] = 1
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Determination', 'compliance_detail'] = 'Compliant by no fine'
    # Fine Waived by Admission
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Admission', 'compliance'] = 1
    blight_df.loc[blight_df['disposition'] == 'Responsible (Fine Waived) by Admission', 'compliance_detail'] = 'Compliant by no fine'
    ## Compliant by Payment
    # Payment with PENDING hearing
    blight_df.loc[(
        (blight_df['hearing_date'] == 'PENDING') &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        (blight_df['hearing_date'] == 'PENDING') &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by payment with PENDING hearing'
    # Transforms values in 'hearing_date' and 'payment_date' to panda date types
    dummy_hearing_date = blight_df['hearing_date'].copy()
    blight_df.loc[blight_df['hearing_date'] == 'PENDING', 'hearing_date'] = np.nan
    blight_df.loc[:, 'hearing_date'] = pd.to_datetime(blight_df['hearing_date'].str.extract('([0-9]{2}/[0-9]{2}/20{1}[0-9]{2})'))
    # Early Payment, payment before hearing date
    blight_df.loc[(
        (blight_df['payment_date'] < blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        (blight_df['payment_date'] < blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by early payment'
    # Payment on time, payment on hearing date
    blight_df.loc[(
        (blight_df['payment_date'] == blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        (blight_df['payment_date'] == blight_df['hearing_date']) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by payment on time'
    # Payment within one month after hearing date
    blight_df.loc[(
        ((blight_df['payment_date'] - blight_df['hearing_date'])/np.timedelta64(1, 'M') <= 1.0) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance'] = 1
    blight_df.loc[(
        ((blight_df['payment_date'] - blight_df['hearing_date'])/np.timedelta64(1, 'M') <= 1.0) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Compliant by payment within 1 Month'
    # Sets 'hearing_date' to orginal state
    blight_df.loc[:, 'hearing_date'] = dummy_hearing_date
    return blight_df

def non_compliant(blight_df):
    '''
    Compliance = 0
    Tickets that are classified as non-compliant because they made no payment or a payment after one month (late payment).
    Returns a pandas dataframe
    '''
    # Transforms values in 'hearing_date' to panda date types
    dummy_hearing_date = blight_df['hearing_date'].copy()
    blight_df.loc[blight_df['hearing_date'] == 'PENDING', 'hearing_date'] = np.nan
    blight_df.loc[:, 'hearing_date'] = pd.to_datetime(blight_df['hearing_date'].str.extract('([0-9]{2}/[0-9]{2}/20{1}[0-9]{2})'))
    # Non-compliant by late payment
    blight_df.loc[(
        ((blight_df['payment_date'] - blight_df['hearing_date'])/np.timedelta64(1, 'M') > 1.0) &
        (blight_df['payment_amount'] > 0) &
        (blight_df['compliance_detail'].isnull())), 'compliance_detail'] = 'Non-compliant by late payment more than 1 month'
    # Non-compliant by no payment
    blight_df.loc[blight_df['compliance_detail'].isnull(), 'compliance_detail'] = 'Non-compliant by no payment'
    # Sets 'hearing_date' to orginal state
    blight_df.loc[:, 'hearing_date'] = dummy_hearing_date
    return blight_df

def populate_compliance(blight_df):
    '''
    Populates compliance and compliance_details in blight_df with the correct values. 
    Returns a panda dataframe.
    '''
    blight_df = null_compliance(blight_df)
    blight_df = compliant(blight_df)
    blight_df = non_compliant(blight_df)  
    return blight_df

def remove_leakage(blight_df):
    ''' 
    In blight_df removes variables to prevent data leakage and variables with mostly NaNs,
    Returns a panda dataframe
    '''
    blight_df = blight_df[['agency_name', 'inspector_name', 'violator_name','violation_street_number', 'violation_street_name', 
                       'mailing_address_street_name', 'mailing_address_street_name', 'mailing_address_city', 
                       'mailing_address_state', 'mailing_address_zip_code', 'mailing_address_non-usa_code', 
                       'mailing_address_country', 'violation_date', 'hearing_date', 'violation_code', 'violation_description',
                       'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 
                       'judgment_amount', 'violation_latitude', 'violation_longitude', 'compliance']]  
    return blight_df 

def process_blight(blight_path, pre_process):
    '''
    If pre_process is true - Loads the raw blight_violations.csv, cleans the data, computes compliance, removes features with 
    data leakage and saves this dataframe. If pre_process is False, then it loads the pre-processed data.
    Returns a pandas dataframe.
    '''
    if pre_process == False:
        blight_df = load_blight_raw(blight_path)
        blight_df = pre_processing(blight_df)
        blight_df = clean_up(blight_df)
        blight_df = populate_compliance(blight_df)
        blight_df = remove_leakage(blight_df)
        save_blight_data(blight_path, blight_df)
    else:
        blight_df = load_blight(blight_path)
    print("PROCESSED Blight Violations dataset has {} observations with {} features each.".format(*blight_df.shape))        
    return blight_df

def save_blight_data(blight_path, blight_df):
    '''
    Saves the processed blight_df
    Returns nothing.
    '''
    csv_path = os.path.join(blight_path, "Blight_Violations_Processed.csv")
    blight_df.to_csv(csv_path)
    
def load_blight(blight_path):
    '''
    Loads the preprocessed blight_violations.csv
    Returns a pandas dataframe.
    '''
    csv_path = os.path.join(blight_path, "Blight_Violations_Processed.csv")
    blight_df = pd.read_csv(csv_path)
    blight_df.set_index('ticket_id', inplace=True)
    return blight_df

In [5]:
# Loads the raw blight data and pre-processes
#blight_df = process_blight(r"C:\Users\Adrian\Google Drive\Datasets\Blight-Violations", True)
blight_df = process_blight(r"C:\Users\aperez\Google Drive\Datasets\Blight-Violations", True)
# Checks to make sure the data loaded properly
blight_df.info()

PROCESSED Blight Violations dataset has 373844 observations with 26 features each.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 373844 entries, 47056 to 247933
Data columns (total 26 columns):
agency_name                      373844 non-null object
inspector_name                   373844 non-null object
violator_name                    373842 non-null object
violation_street_number          373844 non-null int64
violation_street_name            373779 non-null object
mailing_address_street_name      373841 non-null object
mailing_address_street_name.1    373841 non-null object
mailing_address_city             372028 non-null object
mailing_address_state            371565 non-null object
mailing_address_zip_code         372025 non-null object
mailing_address_non-usa_code     1819 non-null object
mailing_address_country          1830 non-null object
violation_date                   373844 non-null object
hearing_date                     373234 non-null object
violation_code         

Now lets remove unrelevant fields and useless observations -
* Observations where compliance is _NaNs_, these observations provide no use to me
* Features that are too noisey or contain many _NaNs_: inspector_name, violator_name, violation_street_number, violation_street_name, mailing_address_street_name, mailing_address_street_name.1, violation_code, and violation_description

In [9]:
# Drop observations where compliance is NaNs
blight_df.dropna(axis=0, subset=['compliance'], inplace=True)
# Drop useless features
blight_df.drop(axis=1, columns=['inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_street_name', 'mailing_address_street_name.1', 'mailing_address_zip_code', 'mailing_address_non-usa_code',
        'violation_code'], inplace=True)
blight_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251545 entries, 47056 to 359077
Data columns (total 17 columns):
agency_name                251545 non-null object
mailing_address_city       249910 non-null object
mailing_address_state      249495 non-null object
mailing_address_country    1646 non-null object
violation_date             251545 non-null object
hearing_date               250935 non-null object
violation_description      251544 non-null object
disposition                242862 non-null object
fine_amount                251544 non-null object
admin_fee                  251545 non-null object
state_fee                  251545 non-null object
late_fee                   251545 non-null object
discount_amount            251545 non-null object
judgment_amount            251545 non-null float64
violation_latitude         251451 non-null float64
violation_longitude        251451 non-null float64
compliance                 251545 non-null float64
dtypes: float64(4), object(13)
me

## Feature Engineering 

Now, we will construct new features. These are:
* detriot_resident - _True_ if 'mailing_address_city' is Detriot, _False_ otherwise
* michigan_resident - _True_ if 'mailing_address_state' is Michigan, _False_ otherwise
* us_resident - _True_ if 'mailing_address_state' is not null but _False_ if 'mailing_address_country' is not null, _False_ otherwise
* violation_month = the numeric month (1-12) that violation occured on
* days_difference = the number of days between when the violation occured and when the hearing date is

### detriot_resident

First fill "NaN" values with "unknown", then convert all the characters to lower case, and removes all punctuation. Finally use regular expressions to find all strings that start with "det".

In [14]:
# Creates 'detriot_resident' feature, setting all values to False
blight_df['detriot_resident'] = 0
# Fill all "NaN" with "unknown"
blight_df['mailing_address_city'].fillna(value="unknown", inplace=True)
# Convert all characters to lower case
blight_df.loc[:, 'mailing_address_city'] = blight_df['mailing_address_city'].str.lower()
# Removes all punctuation
blight_df.loc[:, 'mailing_address_city'] = blight_df['mailing_address_city'].str.extract('(^[a-z ]+)')
# Refill all "NaN" with "unknown"
blight_df['mailing_address_city'].fillna(value="unknown", inplace=True)
# Finds all strings that start with "det" and set "detriot_resident" to True for these values
blight_df.loc[blight_df['mailing_address_city'].str.contains('(^det)'), 'detriot_resident'] = 1

### michigan_resident
First fill "NaN" values with "un", then convert all the characters to lower case, and removes all punctuation. Finally use regular expressions to find all strings that start with "mi".

In [54]:
# Creates 'michigan_resident' feature, setting all values to False
blight_df['michigan_resident'] = 0
# Fill all "NaN" with "unknown"
blight_df['mailing_address_state'].fillna(value="un", inplace=True)
# Convert all characters to lower case
blight_df.loc[:, 'mailing_address_state'] = blight_df['mailing_address_state'].str.lower()
# Removes all punctuation
blight_df.loc[:, 'mailing_address_state'] = blight_df['mailing_address_state'].str.extract('(^[a-z ]+)')
# Refill all "NaN" with "unknown"
blight_df['mailing_address_state'].fillna(value="un", inplace=True)
# Finds all strings that start with "det" and set "detriot_resident" to True for these values
blight_df.loc[blight_df['mailing_address_state'].str.contains('(^mi)'), 'michigan_resident'] = 1

C:\Users\aperez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
C:\Users\aperez\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


### us_resident
First set to True for all observations that have a known state, then False for all observations that have a non-null value in country

In [84]:
# Creates the 'us_resident' feature
blight_df['us_resident'] = 0 
# Sets 'us_resident' to True if the 'mailing_address_state' is not unknown
blight_df.loc[blight_df['mailing_address_state'] != 'un', 'us_resident'] = 1
# Sets 'us_resident' to False if the 'mailing_address_country' is not unknown
blight_df.loc[~blight_df['mailing_address_country'].isnull(), 'us_resident'] = 0

### violation_date
Convert 'violation_date' to proper date format and extract the numeric month.


In [98]:
# Convert 'violation_date' to proper date format  
blight_df['violation_date'] = pd.DatetimeIndex(blight_df['violation_date']).date
# Extract the numeric month. Save this as 'violation_month'
blight_df['violation_month'] = pd.DatetimeIndex(blight_df['violation_date']).month

### days_difference
Convert 'hearing_date' to proper date format and subtract 'violation_date' from 'hearing_date'. Convert this difference to days.

In [107]:
# Convert 'hearing_date' to proper date format
blight_df['hearing_date'] = pd.DatetimeIndex(blight_df['hearing_date']).date
# Subtract 'violation_date' from 'hearing_date'. Convert this difference to days.
blight_df['days_difference'] = (blight_df['hearing_date'] - blight_df['violation_date'])/np.timedelta64(1, 'D')

In [126]:
blight_df.loc[blight_df['days_difference'] < -100, ['violation_date', 'hearing_date', 'days_difference']]

,violation_date,hearing_date,days_difference
ticket_id,,,
239383,2010-12-08,2010-05-13,-209.0
355322,2016-03-17,2015-11-13,-125.0
299434,2013-12-11,2013-01-30,-315.0
239317,2010-12-21,2010-06-14,-190.0
239243,2010-12-21,2010-03-22,-274.0
268084,2011-12-22,2011-03-17,-280.0
357209,2016-04-07,2015-11-13,-146.0
300023,2013-12-19,2013-02-15,-307.0
299321,2013-12-19,2013-01-25,-328.0


### Violation Description

In [22]:
blight_df.dropna(axis=0, subset=['violation_description'], inplace=True)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
import gensim

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', token_pattern='(?u)\\b\\w\\w\\w+\\b')

# Fit and transform
X = vect.fit_transform(blight_df['violation_description'])

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [27]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, 
        id2word=id_map, passes=25, random_state=34)

In [32]:
ldamodel.show_topics(num_topics=10, num_words=5)

[(0,
  '0.326*"vehicle" + 0.314*"motor" + 0.313*"inoperable" + 0.032*"storage" + 0.003*"premise"'),
 (1,
  '0.468*"rental" + 0.388*"registration" + 0.012*"residential" + 0.011*"defective" + 0.011*"vehicles"'),
 (2,
  '0.232*"public" + 0.229*"free" + 0.228*"sidewalks" + 0.228*"adjoining" + 0.041*"hazardous"'),
 (3,
  '0.105*"hours" + 0.100*"bulk" + 0.100*"time" + 0.100*"designated" + 0.100*"deposited"'),
 (4,
  '0.148*"collection" + 0.108*"private" + 0.084*"containers" + 0.081*"secure" + 0.077*"city"'),
 (5,
  '0.083*"comply" + 0.074*"unlawful" + 0.069*"order" + 0.069*"emergency" + 0.068*"danger"'),
 (6,
  '0.171*"premises" + 0.169*"accumulate" + 0.169*"lie" + 0.169*"allowing" + 0.169*"bulk"'),
 (7,
  '0.086*"vacant" + 0.077*"maintain" + 0.070*"structure" + 0.055*"requirements" + 0.055*"113"'),
 (8,
  '0.221*"growth" + 0.221*"plant" + 0.221*"weeds" + 0.221*"excessive" + 0.019*"failed"'),
 (9,
  '0.068*"containers" + 0.066*"early" + 0.066*"violation" + 0.065*"approved" + 0.065*"remain"')

In [21]:
blight_df.loc[blight_df['violation_description'].isnull(), :]

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_street_name,mailing_address_street_name.1,mailing_address_city,mailing_address_state,mailing_address_zip_code,...,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,violation_latitude,violation_longitude,compliance
ticket_id,,,,,,,,,,,,,,,,,,,,,
329109,Department of Public Works,Maydell Bell,PHILLIP COLE,15707,PREST,GRANDMONT,GRANDMONT,DETROIT,MI,48227,...,Responsible by Default,NaN,$20.00,$10.00,$0.00,$0.00,30.0,42.405392,-83.198039,0.0
